In [1]:
### Use this notebook to generate pgBoost SNP-gene linking features ##

## DEFINE RELEVANT PARAMETERS

# Output folder: define a folder to send output (feature matrix, etc.)
output_folder = "." ## Change to desired folder name

# INPUT FILE PATHS #
# For each dataset/cell type (for which multiome scores are being used as pgBoost input, e.g. K562 cells from dataset 1), provide:
# File paths to linking scores for each constituent method in its respective list (Signac, SCENT, Cicero) [generated using https://github.com/elizabethdorans/E2G_Method_Tutorials]
# File path to called peaks (macs2_peaks.bed)
# Also define a list of dataset-cell type labels, which will be appending to feature names
# Each of these lists should be of the same length, with one entry per set of input scores

# Examples are provided and should be replaced with file paths of interest
signac_files = ["example_input_files/signac_peak_gene_links.tsv"] ### REPLACE/ADD FILEPATHS
scent_files = ["example_input_files/scent_peak_gene_links.tsv"] ### REPLACE/ADD FILEPATHS
cicero_files = ["example_input_files/cicero_peak_gene_links.tsv"] ### REPLACE/ADD FILEPATHS
peaks_files = ["example_input_files/macs2_peaks.bed"] ### REPLACE/ADD FILEPATHS
dataset_celltype_labels = ['dataset1_celltype1'] ### REPLACE/ADD LABELS

# Linking distance: maximum peak-TSS distance for candidate peak-gene links (default is TSS +/- 500kb)
linking_distance_threshold = 500000 ## Recommended parameter

# Promoter distance: maximum distance from a gene's TSS defined as the gene's promoter (default is TSS +/- 1kb)
promoter_distance_threshold = 1000 ## Recommended parameter

In [2]:
## IMPORT PACKAGES

import numpy as np
import pandas as pd
import pybedtools
import os

## LOAD RESOURCES

# Load SNP coordinates (1kG SNPs with MAF count > 5)
snps_1kg = pybedtools.BedTool("resources/SNPs.bed.gz")

# Load TSS coordinates
tss = pd.read_csv("resources/TSS.txt.gz", sep = '\t')
tss['chr'] = 'chr' + tss['chr']

In [3]:
## DEFINE FUNCTIONS 

# Adds a 'peak' column based on chromosome, start, and end columns
def add_peak_column(df, chr_col = 'chr', start_col = 'start', end_col = 'end'):
    df['peak'] = df[chr_col].astype(str) + '-' + df[start_col].astype(str) + '-' + df[end_col].astype(str)
    return df

# Adds chromosome, start, and end columns based on a 'peak' column
def add_bed_columns(df, element_col = "peak"):
    # adjust_start_coords = True when processing peak-gene links from Signac, SCENT, Cicero (start coordinate is +1 from peak coordinates called by macs2)
    prev_cols = df.columns.to_list()
    df[["chr", "start", "end"]] = df[element_col].str.split("-", expand = True)
    df[["start", "end"]] = df[["start", "end"]].astype(int)
    df = df[["chr", "start", "end"] + prev_cols]
    return df

# Adjusts peak start coordinates (constituent methods output starting coordinates +1 relative to called peaks; these are adjusted to merge candidate links)
def adjust_peak_start_coordinates(df, start_adjustment = -1):
    df = add_bed_columns(df)
    df['start'] = df['start'] - 1
    df = add_peak_column(df)
    return df.drop(columns = ['chr', 'start', 'end'])
    
# Read linking score predictions from constituent methods (generated using tutorial code in https://github.com/elizabethdorans/E2G_Method_Tutorials)
def read_links(file, score_column, method):
    links = pd.read_csv(file, sep = "\t", comment='#')
    links = adjust_peak_start_coordinates(links)
    links = links.rename(columns = {score_column: method})
    links = links[['peak', 'gene', method]]
    return links

# Define candidate peak-gene links by intersecting gene windows with called peaks
def get_candidate_peak_gene_links(peaks_bedtool, linking_windows_bedtool):
    candidate_pgl = peaks_bedtool.intersect(linking_windows, wa = True, wb = True).to_dataframe()
    candidate_pgl = candidate_pgl.drop(
        columns = ["score", "strand", "thickStart"])
    candidate_pgl = candidate_pgl.rename(
        columns = {"chrom": "chr", 'name': 'peak', "thickEnd": "gene"})
    return candidate_pgl

# Detect peaks that overlap promoters (these will be excluded from candidate links)
def get_peaks_in_promoters(peaks_bedtool, promoter_windows_bedtool):
    peaks_in_promoters = peaks_bedtool.intersect(promoter_windows, wa = True, wb = True).to_dataframe()['name'].tolist()
    return peaks_in_promoters

# Adds a DF column for distance between peak/SNP and gene
def add_distance_to_gene(df, tss = tss, element_col = 'peak', gene_col = 'gene'):
    if 'chr' not in df.columns:
        df = add_bed_columns(df, element_col = element_col)
    df = df.merge(tss, on = [gene_col, "chr"]).copy()
    df['loc'] = np.ceil(df[['start', 'end']].mean(axis = 1))
    df['%s_gene_distance' % element_col] = abs(df["loc"] - df["tss"]).astype(int)
    return df.drop(columns = ['chr', 'start', 'end', 'loc', 'tss'])

# Converts peak-gene links to SNP-gene links by intersecting peaks with SNPs
def peak_gene_links_to_snp_gene_links(peak_gene_links, snp_bed):
    pgl = peak_gene_links[["peak", "gene"]].drop_duplicates()
    pgl = add_bed_columns(pgl)
    sgl = pybedtools.BedTool.from_dataframe(pgl[["chr", "start", "end", "peak", "gene"]]).intersect(
        snp_bed).to_dataframe().rename(columns = {"score": "gene", "name": "peak"})
    if len(sgl) == 0:
        sgl = pd.DataFrame(columns = ['snp', 'peak', 'gene'])
    else:
        sgl["snp"] = sgl["chrom"].astype(str) + "-" + sgl["start"].astype(str) + "-" + sgl["end"].astype(str)
    sgl = sgl[["snp", "peak", "gene"]].merge(peak_gene_links, how = "left", on = ["peak", "gene"])
    return sgl

# Filters peak-gene links by distance to TSS (< 500kb and >1kb, by default, see parameters defined above)
def process_links(df, peaks_in_promoter, linking_distance_threshold):
    # Restrict to candidate links within specified distance to TSS
    df = add_distance_to_gene(df, element_col = 'peak')
    df = df[df['peak_gene_distance'] < linking_distance_threshold]
    # Remove candidate links with a peak overlapping a promoter
    df = df[~df['peak'].isin(peaks_in_promoter)]
    return df.drop(columns = 'peak_gene_distance')

# Generates pgBoost features for a single dataset-cell type pair
# Takes the following input files generated by tutorial code from https://github.com/elizabethdorans/E2G_Method_Tutorials: linking predictions from Signac, SCENT, Cicero & bedfile of ATAC peaks
def create_features_single_dataset(signac_file, scent_file, cicero_file, peaks_file, snp_bed):
    # Create Bedtool for peaks called by macs2
    peaks = pd.read_csv(peaks_file, names = ['chr', 'start', 'end'], sep = "\t", header = None, usecols = range(3))
    peaks = add_peak_column(peaks)
    peaks = pybedtools.BedTool.from_dataframe(peaks)

    # Define peaks that overlap a promoter (these will be excluded from analyses) (default: TSS +/- 1kb)
    peaks_in_promoters = get_peaks_in_promoters(peaks, promoter_windows)

    # Define candidate peak-gene links (peaks within the specified distance to TSS, excluding peaks in promoters)
    candidate_peak_gene_links = get_candidate_peak_gene_links(peaks, linking_windows)
    candidate_peak_gene_links = process_links(candidate_peak_gene_links, peaks_in_promoters, linking_distance_threshold)
    
    print("%s candidate peak-gene links" % len(candidate_peak_gene_links))
    
    # Define SNP-gene links by intersecting peak-gene links with SNPs
    candidate_snp_gene_links = peak_gene_links_to_snp_gene_links(candidate_peak_gene_links, snps_1kg)
    print("%s candidate SNP-gene links" % len(candidate_snp_gene_links))

    # Read linking scores from constituent methods
    signac_links = read_links(signac_file, score_column = "Score", method = "signac")
    signac_links = peak_gene_links_to_snp_gene_links(signac_links, snp_bed)
    scent_links = read_links(scent_file,  score_column = "beta", method = "scent")
    scent_links = peak_gene_links_to_snp_gene_links(scent_links, snp_bed)
    cicero_links = read_links(cicero_file,  score_column = "Score", method = "cicero")
    cicero_links = peak_gene_links_to_snp_gene_links(cicero_links, snp_bed)
    
    # Merge all constituent scores with candidate SNP-gene links
    for links in [signac_links, scent_links, cicero_links]:
        candidate_snp_gene_links = candidate_snp_gene_links.merge(links, on = ['snp', 'peak', 'gene'], how = 'left')
    print(candidate_snp_gene_links)
        
    return candidate_snp_gene_links


In [4]:
## DEFINE CANDIDATE LINKS

# Define windows around TSS for candidate peak-gene links (e.g. <500kb)
linking_windows = tss.copy()
linking_windows["start"] = (linking_windows["tss"] - linking_distance_threshold).clip(0)
linking_windows["end"] = linking_windows["tss"] + linking_distance_threshold
linking_windows = linking_windows[["chr", "start", "end", "gene"]]
linking_windows = pybedtools.BedTool.from_dataframe(linking_windows)

# Define windows around TSS for promoters (e.g. <1kb)
promoter_windows = tss[['chr', 'tss', 'gene']].copy()
promoter_windows['start'] = (tss['tss'] - promoter_distance_threshold).clip(0)
promoter_windows['end'] = tss['tss'] + promoter_distance_threshold
promoter_windows = promoter_windows[['chr', 'start', 'end', 'gene']]
promoter_windows = pybedtools.BedTool.from_dataframe(promoter_windows)


In [ ]:
# For each dataset/cell type, read in all constituent scores and combine to form feature dataframe
features = pd.DataFrame(columns = ['snp', 'gene'])
if len(signac_files) > 0 and (len(signac_files) == len(scent_files) == len(cicero_files) == len(peaks_files) == len(dataset_celltype_labels)):
    inputs = zip(signac_files, scent_files, cicero_files, peaks_files, dataset_celltype_labels)
    for signac_file, scent_file, cicero_file, peaks_file, dataset_celltype_label in inputs:
        print("\n", dataset_celltype_label)
        candidate_sgl = create_features_single_dataset(signac_file, scent_file, cicero_file, peaks_file, snps_1kg)
        renamed_columns = ["%s_%s" % (col, dataset_celltype_label) if col not in ['snp', 'gene'] else col for col in candidate_sgl.columns]
        candidate_sgl.columns = renamed_columns
        # Merge with features dataframe
        features = features.merge(candidate_sgl, on = ['snp', 'gene'], how = 'outer')
else:
    print("Input lists must not be empty, and must be of equal length!")


In [6]:
## ADD DISTANCE-BASED FEATURES

# SNP-gene distance
features = add_distance_to_gene(features, element_col = 'snp')

# Closest TSS (binary)
closest_tss = features.sort_values(by = ["snp", "snp_gene_distance"]
                                 ).drop_duplicates(subset = "snp", keep = "first"
                                 )[["snp", "gene"]]
closest_tss["closest_tss"] = 1

features = features.merge(closest_tss, how = "left", on = ["snp", "gene"])
features["closest_tss"] = features["closest_tss"].fillna(0)
features["closest_tss"] = features["closest_tss"].astype(int)

# Add 'chr' column (for LOCO training)
features['chr'] = features['snp'].str.split("-").str[0]

In [ ]:
## SAVE OUTPUT (feature matrix and column lists)

# Classify columns
drop_duplicates_columns = []
feature_columns = []
for column in features.columns:
    if column in ['snp', 'gene', 'chr']:
        continue
    elif column.startswith("peak"):
        drop_duplicates_columns.append(column)
    else:
        feature_columns.append(column)

if output_folder != None:
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    print("Outputting %s features for %s candidate SNP-gene links!" % (len(feature_columns), len(features)))       
    
    # Save feature matrix (--data_file argument to pgBoost.R)
    features.to_csv("%s/data.tsv" % output_folder, sep = '\t', index = False)
    
    # Save features list (--predictor_file argument to pgBoost.R)
    with open("%s/predictors.txt" % output_folder, 'w') as features_file:
        features_file.write("\n".join(feature_columns))
        
    # Save drop duplicates columns (--drop_duplicates_file argument to pgBoost.R)
    with open("%s/drop_duplicates.txt" % output_folder, 'w') as drop_duplicates_file:
        drop_duplicates_file.write("\n".join(drop_duplicates_columns))
